In [492]:
%run setup_path.py
%load_ext autoreload
%autoreload 2

from os import path

from gorbslam.orbslam_processor import ModelType, ORBSLAMProcessor

# TRAJECTORY_NAME = 'vabadussild'
# TRAJECTORY_NAME = 'vabadusepst'
# TRAJECTORY_NAME = 'oldtown'
# TRAJECTORY_NAME = 'vabadussild_novatel'
TRAJECTORY_NAME = 'vabadusepst_novatel'
# TRAJECTORY_NAME = 'oldtown_novatel'
# TRAJECTORY_NAME = 'kitti/09_30_drive_0018'

# model_type = ModelType.NN
# model_type = ModelType.NN_CLR
model_type = ModelType.GBR
# model_type = ModelType.SVR
# model_type = ModelType.RFR
# model_type = ModelType.UMEYAMA

processor = ORBSLAMProcessor(path.join('~/orbslam-out', TRAJECTORY_NAME), model_type)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [493]:
# processor.initialize_model(overwrite=True)
processor.initialize_model()
processor.fit_trajectories()
processor.save_trajectories()


In [496]:
# processor.model.model.summary()


In [1]:
import plotly.express as px

ape = processor.calculate_ape_metric_all()

fig = px.histogram(x=ape[0].error, marginal="violin")

fig.show()


In [499]:
processor.create_ape_plot_all().show()


In [500]:
# processor.create_ape_plot().show()


In [501]:
processor.create_ape_plot(0).show()


Min APE: 0.009429951139347671
Max APE: 13.585173392228254


In [502]:
processor.create_2d_plot_utm().show()


In [503]:
processor.create_2d_plot_slam().show()


In [504]:
processor.create_map_plot().show()


In [505]:
# processor.create_3d_plot_slam().show()
